In [3]:
import numpy as np
import pandas as pd
import os as os

In [10]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float,
              'floors':str,'condition':int, 'lat':float, 'date':str,
               'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
wrk_dir='C:\Prasanna Krishna\Prasanna Krishna\Analytics\Python\datasets\housing'
os.chdir(wrk_dir)

In [61]:
train=pd.read_csv('kc_house_train_data.csv',dtype=dtype_dict)
test=pd.read_csv('kc_house_test_data.csv',dtype=dtype_dict)
sales=pd.read_csv('kc_house_data.csv',dtype=dtype_dict)

In [66]:
def get_numpy_data(df, features, output):
    #print(df.head())
    df['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    df_numpy=df[features].values
    op_numpy=df[output].values
    #print("sri hari")
    #print(df_numpy)
    return(df_numpy, op_numpy)

In [67]:
df_numpy, op_numpy=get_numpy_data(train, ['sqft_living'], 'price') 
print(df_numpy)
print(op_numpy[0])

[[1.00e+00 1.18e+03]
 [1.00e+00 2.57e+03]
 [1.00e+00 7.70e+02]
 ...
 [1.00e+00 1.53e+03]
 [1.00e+00 1.60e+03]
 [1.00e+00 1.02e+03]]
221900.0


In [57]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix, weights)
    return(predictions)

In [58]:
test_predictions = predict_output(df_numpy, my_weights)
print(test_predictions[0]) # should be 1181.0
print(test_predictions[1]) # should be 2571.0

1181.0
2571.0


In [59]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2*np.dot(errors, feature)
    return(derivative)

In [68]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 

errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative

-23345850016.0
-23345850016.0


In [76]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)


In [123]:
simple_features = ['sqft_living','sqft_living15']
my_output = 'price'
feature_matrix, output=get_numpy_data(train, simple_features, my_output)
initial_weights = np.array([-100000., 1.,1.])
step_size = 7e-12
tolerance = 1.5e5

In [124]:

def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
 
    while not converged:
        predictions=predict_output(feature_matrix, weights)
        #print(predictions)
        errors=predictions - output 
        print('errors',errors)
        gradient_sum_squares = 0 # initialize the gradient sum of squares 
        for i in range(len(weights)): # loop over each weight
            
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            feature =feature_matrix[:,i]
            #print('feature',feature)
            derivative = feature_derivative(errors, feature)
            # add the squared value of the derivative to the gradient magnitude (for assessing convergence)
            gradient_sum_squares = gradient_sum_squares +  derivative **2
            # subtract the step size times the derivative from the current weight
            #print("derivative",derivative)
            #print(weights[i])
            weights[i] =  weights[i] - step_size* derivative
            # compute the square-root of the gradient sum of squares to get the gradient matnigude:
            gradient_magnitude = sqrt(gradient_sum_squares)
            
        if gradient_magnitude < tolerance:
            print("errors====>",errors)
            print("gradient_magnitude",gradient_magnitude)
            converged = True
        return(weights)

In [125]:
simple_weights = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print(simple_weights)


errors [-319380. -633740. -276510. ... -456940. -496990. -422960.]
[-99999.84538308    380.56901949    343.87581535]


In [126]:
df_numpy, op_numpy=get_numpy_data(train, ['sqft_living','sqft_living15'], 'price') 
train_predictions=predict_output(df_numpy, simple_weights)
rss=(((train_predictions-train['price'])**2).sum()/train.shape[0])**(1./2)

In [127]:
 print('training rss',rss)

training rss 924131.901542456


In [128]:
 print('training rss',rss)

training rss 924131.901542456
